# Note-Saving Tool with Pydantic and Anthropic Tool Use

In this example, we'll create a tool that saves a note with the author and metadata, and use Pydantic to validate the model's response when calling the tool. We'll define the necessary Pydantic models, process the tool call, and ensure that the model's response conforms to the expected schema.

## Step 1: Set up the environment
First, let's install the required libraries and set up the Anthropic API client.

In [ ]:
%pip install -qU boto3 pydantic 'pydantic[email]'

In [1]:
import boto3
from pydantic import BaseModel, EmailStr, Field
from typing import Optional

client = boto3.client("bedrock-runtime", region_name="us-west-2")
MODEL_NAME = "anthropic.claude-3-sonnet-20240229-v1:0"

## Step 2: Define the Pydantic models

We'll define Pydantic models to represent the expected schema for the note, author, and the model's response. This will allow us to validate and type-check the model's response when saving a note.

In [2]:
class Author(BaseModel):
    name: str
    email: EmailStr

class Note(BaseModel):
    note: str
    author: Author
    tags: Optional[list[str]] = None
    priority: int = Field(ge=1, le=5, default=3)
    is_public: bool = False

class SaveNoteResponse(BaseModel):
    success: bool
    message: str

## Step 3: Define the client-side tool

Next, we'll define the client-side tool that our chatbot will use to save notes.

In [16]:
tools = [
    {
        "toolSpec": {
            "name": "save_note",
            "description": "A tool that saves a note with the author and metadata.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "note": {
                            "type": "string",
                            "description": "The content of the note to be saved."
                        },
                        "author": {
                            "type": "object",
                            "properties": {
                                "name": {
                                    "type": "string",
                                    "description": "The name of the author."
                                },
                                "email": {
                                    "type": "string",
                                    "format": "email",
                                    "description": "The email address of the author."
                                }
                            },
                            "required": ["name", "email"]
                        },
                        "priority": {
                            "type": "integer",
                            "minimum": 1,
                            "maximum": 5,
                            "default": 3,
                            "description": "The priority level of the note (1-5)."
                        },
                        "is_public": {
                            "type": "boolean",
                            "default": False,
                            "description": "Indicates whether the note is publicly accessible."
                        }
                    },
                    "required": ["note", "author"]
                }
            }
        }
    }
]

## Step 4: Implement the note-saving tool
We'll create a dummy note saving function that just prints out that the note was saved successfully. If you actually want this note to be saved somewhere, you can implement this function.

In [4]:
def save_note(note: str, author: dict, priority: int = 3, is_public: bool = False) -> None:
    print("Note saved successfully!")

## Step 5: Process the tool call and generate the response
We'll create functions to process the tool call made by Claude and generate the response indicating the success of saving the note.

In [5]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "save_note":
        note = Note(
            note=tool_input["note"],
            author=Author(
                name=tool_input["author"]["name"],
                email=tool_input["author"]["email"]
            ),
            priority=tool_input.get("priority", 3),
            is_public=tool_input.get("is_public", False)
        )
        save_note(note.note, note.author.model_dump(), note.priority, note.is_public)
        return SaveNoteResponse(success=True, message="Note saved successfully!")

def generate_response(save_note_response):
    return f"Response: {save_note_response.message}"

## Step 6: Interact with the chatbot

Now, let's create a function to interact with the chatbot. We'll send a user message, process the tool call made by Claude, generate the response, validate the model's response using Pydantic, and return the final response to the user.

In [65]:
def chatbot_interaction(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "text": user_message
                }
            ]
        }
    ]

    message = client.converse(
        modelId=MODEL_NAME,
        inferenceConfig={
            "temperature": 0,
            "maxTokens": 4000
        },
        toolConfig={
            'tools': tools
        },
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message['stopReason']}")
    print(f"Content: {message['output']['message']['content']}")

    if message['stopReason'] == "tool_use":
        tool_use = next(block for block in message['output']['message']['content'] if "toolUse" in block )
        tool_name = tool_use['toolUse']['name']
        tool_input = tool_use['toolUse']['input']

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        save_note_response = process_tool_call(tool_name, tool_input)


        print(f"Tool Result: {save_note_response}")

        response = client.converse(
            modelId=MODEL_NAME,
            inferenceConfig={
                "temperature": 0,
                "maxTokens": 4000
            },
            messages=[
                {
                    "role": "user",
                    "content": [{
                        "text": user_message
                    }]
                },
                {
                    "role": "assistant",
                    "content": message['output']['message']['content']
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use['toolUse']['toolUseId'],
                                "content": [{
                                    "text": str(save_note_response),
                                }]
                            }
                        }
                    ],
                },
            ],
            toolConfig={
                'tools': tools
            },
        )
    else:
        response = message

    final_response = next(
        (block['text'] for block in response['output']['message']['content'] if "text" in block)
        ,None,
    )
    print(response['output']['message']['content'])
    print(f"\nFinal Response: {final_response}")

    return final_response

## Step 7: Test the chatbot
Let's test our chatbot with a sample query to save a note.

In [66]:
chatbot_interaction("""
Can you save a private note with the following details?
Note: Remember to buy milk and eggs.
Author: John Doe (johndoe@gmail.com)
Priority: 4
""")


User Message: 
Can you save a private note with the following details?
Note: Remember to buy milk and eggs.
Author: John Doe (johndoe@gmail.com)
Priority: 4


Initial Response:
Stop Reason: tool_use
Content: [{'text': 'Here is how we can save the note with the provided details:'}, {'toolUse': {'toolUseId': 'tooluse_0fOzh3ZASFqVxAHIKigeAw', 'name': 'save_note', 'input': {'note': 'Remember to buy milk and eggs.', 'author': {'name': 'John Doe', 'email': 'johndoe@gmail.com'}, 'priority': 4, 'is_public': False}}}]

Tool Used: save_note
Tool Input: {'note': 'Remember to buy milk and eggs.', 'author': {'name': 'John Doe', 'email': 'johndoe@gmail.com'}, 'priority': 4, 'is_public': False}
Note saved successfully!
Tool Result: success=True message='Note saved successfully!'
[{'text': 'Great, the note "Remember to buy milk and eggs." has been successfully saved as a private note with priority 4 for the author John Doe (johndoe@gmail.com).'}]

Final Response: Great, the note "Remember to buy milk

'Great, the note "Remember to buy milk and eggs." has been successfully saved as a private note with priority 4 for the author John Doe (johndoe@gmail.com).'

In this example, we've created a tool that saves a note with the author and metadata. The chatbot uses the save_note tool to save the note, and Pydantic is used to validate the model's response when calling the tool. The Note, Author, and SaveNoteResponse models ensure that the tool input and the model's response conform to the expected schema.

By defining clear Pydantic models and using them to validate the model's response, we add an extra layer of reliability and safety when interacting with the chatbot and saving notes.